<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/translator_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#번역 모델(helsinki) 학습 및 추론 코드

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00


In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset, DatasetDict
import pickle
import torch
import pandas as pd
import os

#데이터 병합 및 토크나이저, 처리(필요시 사용)

In [7]:

# # 3. 데이터셋 로드 (예시 데이터셋)
# df = pd.read_csv('/content/drive/MyDrive/translator/data/train.csv')#train.csv파일 있으면 루트 변경
# dataset = Dataset.from_pandas(df)


In [8]:
# # combined_df를 Dataset으로 변환
# dataset = Dataset.from_pandas(df)

# # train, test 데이터셋 분리 (예시로 10%를 테스트 세트로 분리)
# train_test = dataset.train_test_split(test_size=0.1)
# datasets = DatasetDict({
#     'train': train_test['train'],
#     'test': train_test['test']
# })

# # 4. 전처리 함수 (한국어 -> 영어 번역)
# def preprocess_function(examples):
#     inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
#     targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     # 타겟 텍스트도 토크나이징
#     labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # # 데이터셋을 전처리 및 토크나이징 (datasets가 이미 로드된 상태에서)
# tokenized_datasets = datasets.map(preprocess_function, batched=True)


In [9]:
#토크나이징된 데이터셋을 4개로 나눔


# # 5. 4개의 파트로 분할하기 위해 총 데이터셋 크기 계산
# dataset_length = len(tokenized_datasets['train'])

# #파일을 4개로 나누어서 저장, 나중에 각각 불러와서 에폭마다 돌려야할듯
# split_size = dataset_length // 4

# # 7. 각 파트를 pkl 파일로 저장
# for i in range(4):
#     start_idx = i * split_size
#     end_idx = (i + 1) * split_size if i < 3 else dataset_length  # 마지막 파트는 끝까지

#     part_dataset = tokenized_datasets['train'].select(range(start_idx, end_idx))

#     output_file = f"/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_{i+1}.pkl"
#     with open(output_file, 'wb') as f:
#         pickle.dump(part_dataset, f)

#     print(f"Tokenized dataset part {i+1} saved to {output_file}")

#학습 세팅(토크나이징 된 데이터 받음)


In [10]:
#모델과 데이터셋 설정
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/815k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/959k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/418M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:

# 4. 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/translator/model",  # 모델 및 체크포인트가 저장될 디렉토리
    evaluation_strategy="steps",  # 매 스텝마다 평가
    save_strategy="steps",  # 스텝마다 체크포인트 저장
    save_steps=500,  # 500 스텝마다 체크포인트 저장
    save_total_limit=3,  # 최근 3개의 체크포인트만 유지
    logging_steps=100,  # 로그를 출력할 스텝 간격
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # 각 파일별로 5 에폭씩 학습
    weight_decay=0.01,
    load_best_model_at_end=True,  # 학습 종료 시 가장 성능이 좋은 모델 불러오기
    save_on_each_node=False,  # 모든 노드에서 체크포인트를 저장하지 않음 (분산 학습 시)
    logging_dir="/content/drive/MyDrive/translator/logging",  # 로그를 저장할 디렉토리
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
model_checkpoint_path = "/content/drive/MyDrive/translator/checkpoints"

In [13]:
last_checkpoint = None
if os.path.isdir(model_checkpoint_path) and os.listdir(model_checkpoint_path):
    last_checkpoint = Trainer.get_last_checkpoint(model_checkpoint_path)
    if last_checkpoint is not None:
        print(f"체크포인트에서 이어서 학습을 시작합니다: {last_checkpoint}")
    else:
        print("체크포인트가 없습니다. 새로 학습을 시작합니다.")
else:
    print("체크포인트 디렉토리가 존재하지 않습니다. 새로 학습을 시작합니다.")


체크포인트 디렉토리가 존재하지 않습니다. 새로 학습을 시작합니다.


#학습(학습 파일은 여러개로 되어있음)

In [14]:
# 3. 데이터셋 리스트
file_paths = [
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_1.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_2.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_3.pkl",
    "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_4.pkl"
]

In [15]:
def load_preprocessed_data(file_path):
    with open(file_path, 'rb') as f:
        tokenized_datasets = pickle.load(f)
    return tokenized_datasets

In [ ]:
for i, file_path in enumerate(file_paths):
    # 각 파트의 데이터 불러오기
    print(file_path)
    print(f"Loading dataset part {i+1} from {file_path}")
    loaded_data = load_preprocessed_data(file_path)

    # 6. 데이터 콜레이터 설정 (Seq2Seq에 맞는 형태)
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # 7. Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=loaded_data,  # 평가 데이터셋 추가
        data_collator=data_collator,
        tokenizer=tokenizer
    )

    # 8. 학습 시작
    print(f"Training on dataset part {i+1}...")
    try:
        trainer.train()
    except KeyboardInterrupt:
        print(f"Training interrupted during part {i+1}. Last checkpoint saved.")

    # api키는 이걸로 넣으면 됨 513a1f0c050fa7f60a76b5232e904d8df397082e

    # 9. 체크포인트 저장
    trainer.save_model(f"/content/drive/MyDrive/translator/model_part_{i+1}")
    print(f"Model checkpoint saved after part {i+1}")


/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_1.pkl
Loading dataset part 1 from /content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_1.pkl
Training on dataset part 1...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("/content/drive/MyDrive/translator/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/translator/final_model")
print("Final model and tokenizer saved.")

#입력 및 예측


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 1. 토크나이저와 모델 로드 (학습된 번역 모델 사용)
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

input_text="what should i say to you?"

inputs = translation_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
translated = translation_model.generate(**inputs)
translated_text = translation_tokenizer.decode(translated[0], skip_special_tokens=True)

# 6. 번역 결과 출력
print(f"번역된 영어 문장: {translated_text}")
